In [ ]:
import pandas as pd

import platform

from config import (
    CITIES_GEO_LOCATION_CSV_PATH,
    BEST_HOTELS_OF_CITIES_JSON_PATH
)

# display settings
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', 1000)          # Expand display width
pd.set_option('display.max_colwidth', None)   # Show full content in each cell

Data folder: /workspaces/jedha-cdsd-cert-projects/bloc-1/kayak/data


In [2]:
if not CITIES_GEO_LOCATION_CSV_PATH.exists():
    print("Please run notebook 01-b1-kayak-destinations.ipynb first!")
else:
    print("Loading cities...")
    cities_df = pd.read_csv(CITIES_GEO_LOCATION_CSV_PATH)
    display(cities_df)

Loading cities...


,id,name,latitude,longitude
0,1,Mont Saint Michel,48.635954,-1.511460
1,2,St Malo,48.649518,-2.026041
2,3,Bayeux,49.276462,-0.702474
3,4,Le Havre,49.493898,0.107973
4,5,Rouen,49.440459,1.093966
5,6,Paris,48.858890,2.320041
6,7,Amiens,49.894171,2.295695
7,8,Lille,50.636565,3.063528
8,9,Strasbourg,48.584614,7.750713
9,10,Chateau du Haut Koenigsbourg,48.249411,7.344320


In [3]:
# We can clear existing file to do a fresh scrapping
# BEST_HOTELS_OF_CITIES_JSON_PATH.unlink(missing_ok=True)

if not BEST_HOTELS_OF_CITIES_JSON_PATH.exists():
    print("Scrapping best hotels of cities...")

    for city in cities_df.itertuples():
        if platform.system() == 'Windows':
            run=f"python scrapper/src/scrapper/main.py --city-id \"{city.id}\" --name \"{city.name}\" --latitude \"{city.latitude}\" --longitude \"{city.longitude}\" --json-store-file-path \"{BEST_HOTELS_OF_CITIES_JSON_PATH}\""
        else:
            run=f"python scrapper/src/scrapper/main.py --city-id '{city.id}' --name '{city.name}' --latitude '{city.latitude}' --longitude '{city.longitude}' --json-store-file-path '{BEST_HOTELS_OF_CITIES_JSON_PATH}'"
    
        !{run}
else:
    print("Best hotels already scrapped.")




Scrapping best hotels of cities...
Scrapping hotels for destination: Mont Saint Michel (id: 1, latitude: 48.6359541, longitude: -1.51146)
INFO  [25-04-24 20:07:13] scrapy.utils.log log_scrapy_info :173 Scrapy 2.12.0 started (bot: scrapybot)
INFO  [25-04-24 20:07:13] scrapy.utils.log log_scrapy_info :182 Versions: lxml 5.3.2.0, libxml2 2.12.10, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Mar 18 2025, 03:29:18) [GCC 12.2.0], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.2, Platform Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.36
INFO  [25-04-24 20:07:13] scrapy.addons load_settings :48 Enabled addons:
[]
DEBUG [25-04-24 20:07:13] asyncio __init__ :54 Using selector: EpollSelector
DEBUG [25-04-24 20:07:13] scrapy.utils.log log_reactor_info :188 Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
DEBUG [25-04-24 20:07:13] scrapy.utils.log log_reactor_info :192 Using asyncio event loop: asyncio.unix_e

## Hotels preview

Preview number of hotels per city

In [ ]:
from utils import read_df_from_jsonlines

hotels = read_df_from_jsonlines(BEST_HOTELS_OF_CITIES_JSON_PATH)

hotels_count = hotels[["city_id", "name"]].groupby(["city_id"]).count().reset_index()
hotels_count.columns = ["city_id", "hotels_count"]

cities_hotels_stats_df = cities_df.merge(hotels_count, left_on="id", right_on="city_id") # , suffixes=(None, "_count")

print(cities_hotels_stats_df)

    id                          name   latitude  longitude  city_id  hotels_count
0    1             Mont Saint Michel  48.635954  -1.511460        1            20
1    2                       St Malo  48.649518  -2.026041        2            20
2    3                        Bayeux  49.276462  -0.702474        3            20
3    4                      Le Havre  49.493898   0.107973        4            20
4    5                         Rouen  49.440459   1.093966        5            20
5    6                         Paris  48.858890   2.320041        6            20
6    7                        Amiens  49.894171   2.295695        7            20
7    8                         Lille  50.636565   3.063528        8            20
8    9                    Strasbourg  48.584614   7.750713        9            20
9   10  Chateau du Haut Koenigsbourg  48.249411   7.344320       10            20
10  11                        Colmar  48.077752   7.357964       11            20
11  12          